# Practicing Machine Learning with Decision Tree 

In our first notebook, we looked at how to create a Naïve Bayes classifier. In the `R` notebook, we were introduced to a Decision Tree. Today, we will be working with a Decision Tree again, but this time with `Python` and `sklearn`. This is an appropriate time to bring up that these two classification techniques are part of a larger family of machine learning algorithms known as supervised learning. This means that some human intervention is needed in order to create an optimal model. In order to do this, we have to create training and testing sets to evaluate our model's performance and prune based on this performance.   

In [1]:
import pandas as pd
import numpy as np 
from sklearn import datasets

We are going to continue using the `iris` dataset for the practice, but remember, reading in the dataset wasn't as straight forward as reading it in from a file. We will go ahead and do this for you again.

In [2]:
iris = datasets.load_iris() # load the iris dataset from sklearn
data = pd.DataFrame(data=iris.data) # create frame of input data
target = pd.DataFrame(data=iris.target) # create frame of target data

df = pd.concat((data,target), axis=1) # combine input and target together
col_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'] # column names
df.columns = col_names # name data frame columns

df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


I wasn't happy with the values for `species` in the lab. Although keeping the values 0, 1, and 2 are fine and won't interfere with the classification task, it isn't very reader friendly. But one can argue that the species names are kind of long and perhaps we can opt for an abbreviated, albeit readable value instead.

**Activity 1**: *Replace the values of the `species` column where 0 will become 'S', 1 will become 'VS', and 2 will become 'VG'.*

In [3]:
# Code for Activity 1 goes here
# *****************************

vals_to_replace = {0:'setosa', 1:'versicolor', 2:'verginica'}
df['species'] = df['species'].map(vals_to_replace)

Since we are using a Decision Tree method this time, we have to import a new module from `sklearn`. This module is called `tree`.

In [4]:
from sklearn import tree

Last time we divided the data based on inputs and target over the entire dataset and then used the same dataset to assess the accuracy of the model. In practice, this is never a good idea because you could be training your model specifically to the sample that you have, a phenomenon known as over-fitting. 

Instead, to avoid this, we want to divide our dataset into a training set and a testing set. The training set is what we train our model on. Makes sense. And the test set is what we evaluate the performance of our model on. 

Now, we of course want to take a random sampling of our original dataset to be our training set and everything else will be our testing set. Here is how we sample a data frame...

In [5]:
df.sample(n = 5, random_state = 1)

,sepal_length,sepal_width,petal_length,petal_width,species
14,5.8,4.0,1.2,0.2,setosa
98,5.1,2.5,3.0,1.1,versicolor
75,6.6,3.0,4.4,1.4,versicolor
16,5.4,3.9,1.3,0.4,setosa
131,7.9,3.8,6.4,2.0,verginica


Okay, notice how `sample()` is a method of data frame objects. Calling this on our data frame `df` will randomly sample rows and return a specified number `n`. Here we chose to return only 5 randomly sampled rows. There is also a parameter called `random_state` that we are using here. We chose to pass it the argument `1`. What this does is make this random sample replicable so that if you ran this line again then the exact same sample would come up. 

Normally, the majority of the observations will serve as your training set. The `iris` dataset isn't very large (only 150 observations) so we will take only 2/3rds (100 observations) of our dataset to train our model.



**Activity 2**: *Sample 100 rows from the `df` data frame and call this newly created data frame **`train`**. Be sure to specify `random_state = 1`.*

In [8]:
# Code for Activity 2 goes here
# *****************************

train = df.sample(n = 100, random_state = 1)
train

,sepal_length,sepal_width,petal_length,petal_width,species
14,5.8,4.0,1.2,0.2,setosa
98,5.1,2.5,3.0,1.1,versicolor
75,6.6,3.0,4.4,1.4,versicolor
16,5.4,3.9,1.3,0.4,setosa
131,7.9,3.8,6.4,2.0,verginica
56,6.3,3.3,4.7,1.6,versicolor
141,6.9,3.1,5.1,2.3,verginica
44,5.1,3.8,1.9,0.4,setosa
29,4.7,3.2,1.6,0.2,setosa
120,6.9,3.2,5.7,2.3,verginica


Now we need to take those rows from `df` that were not in the training set and use them as our testing set. 

**Activity 3** *Create a testing set by **`drop`**ping **`train.index`** from **`df`**. Call this new frame **`test`**. Hint: all of the bolded words should be used in your line of code.*  

In [9]:
# Code for Activity 3 goes here
# *****************************

train = df.sample(n = 100, random_state = 1)
test = df.drop(train.index)

If you were having trouble with the above 2 pieces of code, below is what you should have.

In [29]:
train = df.sample(n = 100, random_state = 1)
test = df.drop(train.index)

Now we have to split the data by the inputs and the target for the training set so we can begin defining the model. To do that, we need to change the inputs (all columns except for the `species` column) to a `numpy` array. Then we can define the target by converting `train.species` into a `numpy` array as well. Here is how you do that below...

In [30]:
train_X = np.asarray(train[['sepal_length','sepal_width','petal_length','petal_width']])
train_y = np.asarray(train.species)

There we go! Now we have our training data in a state that is ready to train our model!

Now we don't need our testing data in this format for a few lines, but let's be prepared...

**Activity 4**: *Split the **`test`** data frame into inputs and target. Call the input columns **`test_X`** and the target column **`test_y`**.* 

In [31]:
# Code for Activity 4 goes here
# *****************************

test_X = np.asarray(test[['sepal_length','sepal_width','petal_length','petal_width']])
test_y = np.asarray(test.species)

Now we are ready to train the model. This should look very similar to our Naïve Bayes experience in `Python`. But remember, this time we are training our model on our training data.

In [32]:
clf = tree.DecisionTreeClassifier(criterion='entropy')
clf = clf.fit(train_X, train_y )

Now we can see how our model did by assessing it with the data that trained it. We did a similar thing in the lab notebook, except with the entire data set. Let's do this again to see how well it classifies on the dataset that defined it... 

In [33]:
y_pred = clf.fit(train_X, train_y).predict(train_X)
print("Number of mislabeled points out of a total {} points : {}"
      .format(len(train),(train_y != y_pred).sum()))

Number of mislabeled points out of a total 100 points : 0


EUREKA! A perfect classifier! Wait just a moment...we are testing on our training set. That isn't that impressive after all. What we really need to do is assess how well it predicts the testing set.

**Activity 5**: *Using the template above, use the model trained with the training set and test its performance with the testing set. Remember, the only part that should stay the same as above is the `clf.fit(train_X, train_y)`. Everything else should be using the data from the testing set.*

In [35]:
# Code for Activity 5 goes here
# *****************************


y_pred = clf.fit(train_X, train_y).predict(test_X)
print("Number of mislabeled points out of a total {} points : {}"
      .format(len(test),(test_y != y_pred).sum()))

Number of mislabeled points out of a total 50 points : 6


If everything went correctly, we should see that our model misclassified 5 out of 50 points.  All in all, not too bad.

Now, there is one more thing that I want to discuss, and that is feature selection. You may have noticed this in `R` and we can find it here too, but the Decision Tree selects the best features to make decisions on and will leave those features out that don't provide much information in predicting our target. We can see that here by calling `feature_imporatances_` on our model `clf`.

For future models, this could actually come in handy when trying to refine our model. There is such thing as too much data, and not all algorithms act like Decision Trees, so often, Decision Trees is a good place to start\* just to see what features are important and which ones to throw out of your model. 

\* *There are other methods that we could use, but we will save those for a later date as they are more mathematically complex.*

In [36]:
z = zip(col_names[0:4],clf.feature_importances_)
list(z)

[('sepal_length', 0.0),
 ('sepal_width', 0.02824956187588366),
 ('petal_length', 0.0),
 ('petal_width', 0.9717504381241163)]

There we have it, we can see that sepal length isn't really providing much information. Petal length provides the most. In the future, we could either set a threshold for the amount of information needed for it to make it into our model in order to refine our model to make better predictions.

### Finding the Right Algorithm

Finally, if you are tripped up on what algorithm to use, `sklearn` has compiled a nice little graphic to help us find the right algorithms given the type of data we have.

<img src="../images/alg_cheat_sheet.png">

